In [ ]:
# Importing package dependencies and storing them as shortcuts "pd" and "np"
import pandas as pd
import numpy as np

In [ ]:
# Importing Dataframe from local with path
dataframe = pd.read_excel('C:\\Users\\sakth\\Downloads\\D149_raw.xlsx')

In [ ]:
# standardizing data by dividing all values by the first value in respective column
for col in dataframe.columns.values.tolist():
    dataframe[col] = dataframe[col] / dataframe[col].loc[0]

In [ ]:
# Dropping unnecessary additional index column (the boldened,unlabeled index column below is the default one)
dataframe.drop(columns=dataframe.columns[0], axis=0, inplace=True)

In [ ]:
# Cell to be used if there are unwanted columns in dataframe to be removed before analysis
# Otherwise, col_names variable will not be used
# col_names_initial = dataframe.columns.values.tolist()
# col_names_remove = ["Row #","Timestamp"] # example row names to be removed
# col_names = list(set(col_names_initial) - set(col_names_remove))

# If using this cell, replace col_names_initial with col_names below

In [ ]:
# Ignore performance related warnings asking to concatenate all columns
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

# Extract column names as a list 
col_names_initial = dataframe.columns.values.tolist()

# Create a new Dataframe to store number of peaks
peaks_df = pd.DataFrame(columns = col_names_initial)

# The code within the indent of this for loop, will run for all columns in Dataframe
for col_name in col_names_initial:
    col_bwd_diff = 'd'+col_name # create new column - backward difference
    col_fwd_diff = 'd2'+col_name # create new column - forward difference
    peaks_col = 'peaks'+col_name # create new column - will store value 1.0, if point is a peak
    
#   For backward difference column, minus value with that of PREVIOUS row/second and vice versa
    dataframe[col_fwd_diff] = dataframe[col_name] - dataframe[col_name].shift(-1) 
    dataframe[col_bwd_diff] = dataframe[col_name] - dataframe[col_name].shift(1)
    
#   Convert all numbers to purely pandas numerics (this ensures they are not misinterpreted as strings)
    dataframe[col_bwd_diff] = pd.to_numeric(dataframe[col_bwd_diff])
    dataframe[col_fwd_diff] = pd.to_numeric(dataframe[col_fwd_diff])
    
#   using a second dataframe just for calculation of threshold, using only first 450 rows (change values accordingly)
    dataframe2 = dataframe.head(450)
    mean_calculated = dataframe2[col_bwd_diff].iloc[350:450].mean() # mean
    std_dev_calculated = dataframe2[col_bwd_diff].iloc[350:450].std() # standard deviation
    threshold_calculated = 4*std_dev_calculated + mean_calculated # threshold, for each column
    
    dataframe[col_name] = pd.to_numeric(dataframe[col_name])
    dataframe[peaks_col] = np.where(
        (
            (dataframe[col_bwd_diff] > threshold_calculated) &
            (
                (dataframe[col_bwd_diff].shift(10) < threshold_calculated)&
                (dataframe[col_bwd_diff].shift(-10) < threshold_calculated) &
                (dataframe[col_bwd_diff].shift(-1) < dataframe[col_bwd_diff])&
                (dataframe[col_bwd_diff].shift(1) < dataframe[col_bwd_diff])
            )
        ),1, 0)
    dataframe = dataframe.drop(columns=[col_bwd_diff,col_fwd_diff])
    if dataframe[dataframe[peaks_col] == 1].size > 0:
        peaks_df.loc[0,col_name] = dataframe[peaks_col].value_counts()[1.0]
    else:
        pass


In [ ]:
# Show peaks dataframe
peaks_df

In [ ]:
# Calculate mean of number of peaks from all columns
# mean_peaks = peaks_df.mean(axis=1)
# mean_peaks

In [ ]:
# Drop all columns in peaks dataframe where there are no peaks
peaks_df.dropna(axis=1, how='all')

In [ ]:
# Check number of peaks for particular column
peaks_df["Mean(ID10827)"]

In [ ]:
# Use to check datapoints where peak occurs for a specific column
# dataframe[["peaksMean(ID10902)"]][dataframe["peaksMean(ID10902)"]==1]

In [ ]:
# check that original dataframe is unaltered
# dataframe

In [ ]:
# import dependencies for plotting graphs, with shortcut
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Plot for a PARTICULAR column, modify accordingly in red highlighted string below i.e., replace "Mean(IDXXXXX)"
sns.lineplot(data = dataframe, x = dataframe.index, y = 'Mean(ID10827)')
sns.lineplot(data = dataframe, x = dataframe.index, y = 'peaksMean(ID10827)')
plt.show()